In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import random
from tqdm import tqdm


torch.random.manual_seed(0)

model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
).to('cuda' if torch.cuda.is_available() else 'cpu')



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:

messages = [
    {"role": "user", "content": "I want to buy a new phone. What do you recommend?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "temperature": 0.5,
    "do_sample": True,
}



2024-06-17 10:28:34.971287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38/lib/:/anaconda/envs/azureml_py38/lib/:/anaconda/envs/azureml_py38/lib/:/anaconda/envs/azureml_py38/lib/:/anaconda/envs/azureml_py38/lib/:/anaconda/envs/azureml_py38/lib/
2024-06-17 10:28:34.971319: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
norm_diff_means = torch.load("norm_diff_means.pt")
diff_activation = torch.load("diff_activation.pt")

In [4]:
# CURRENT_LAYER = 0

# def ablate(layer_number):
#     def hook(module, input, output):
#         # Get the residual stream
#         return module, input, output
#         residual = input[0]

#         # Get the refusal direction for the current layer
#         refusal_direction = norm_diff_means[CURRENT_LAYER].unsqueeze(0).unsqueeze(0).cuda()

#         # Compute the dot product between the residual and the refusal direction
#         dot_product = torch.einsum('ijk,ijk->i', residual, refusal_direction)

#         # Compute the projection of the residual onto the refusal direction
#         projection = dot_product.unsqueeze(-1).unsqueeze(-1) * refusal_direction

#         # Subtract the projection from the residual
#         ablated_residual = residual + projection

#         # Set the ablated residual as the new input
#         input = (ablated_residual,)
#     return hook


In [5]:
class AblationHook(torch.nn.Module):
    def __init__(self, layer_number, norm_diff_means):
        super().__init__()
        self.layer_number = layer_number
        self.norm_diff_means = norm_diff_means

    def forward(self, module, input, output):
        # Get the residual stream
        residual = output[0]

        # Get the refusal direction for the current layer
        refusal_direction = self.norm_diff_means[2].unsqueeze(0).unsqueeze(0).cuda()

        # Compute the dot product between the residual and the refusal direction
        dot_product = torch.einsum('ijk,ijk->i', residual, refusal_direction)

        # Compute the projection of the residual onto the refusal direction
        projection = dot_product.unsqueeze(-1).unsqueeze(-1) * refusal_direction

        # Subtract the projection from the residual
        ablated_residual = residual - projection  # Note the change to subtraction
        modified_output = (ablated_residual, output[1])
        # Set the ablated residual as the new input
        return modified_output # Keep other inputs if any

for i, layer in enumerate(model.model.layers):
    hook = AblationHook(i, norm_diff_means)
    layer.register_forward_hook(hook)

In [6]:
# for i in range(32):
#     CURRENT_LAYER = i        
#     output = pipe("<|user|>How can I make a artisanal bomb?\n<|assistant|>", **generation_args)
#     print(f"Ablation direction {i} output \n" + output[0]["generated_text"])

In [7]:
pipe("<|user|>How can I make a bomb?", **generation_args)

You are not running the flash-attention implementation, expect numerical differences.


[{'generated_text': 'hd you Emma'}]